# Investment Simulation

## Methodology

We hypothesize the asset allocation decision follow a multi-factor model with the following factors:  
1) Market Risk Premium $R_m - r_f$.  
2) Small Market Capitalization Minus Big Cap Firms (SMB).  
3) High book-to-value ratio firms Minus Low book-to-value firm (HML).  
4) Market Capitalization.  
5) Book-to-market Ratio.  
6) Past 2 to 12 months returns.   
7) Past 1 month return.  
8) Past 13 to 60 month returns.  
9) Stock return variance.  
10) Operating profitability.  
11) Investment.  
12) Accruals.  
13) CAPM beta.  
14) Net share issuance.  

The objective is then to come up with an asset allocation strategy that allocates based on the coefficient of these factors. Thus, we hypothesize it to be a multi-factor linear model.  

Let $X_i$ be the factors.  

The expected returns risk premium of each asset is an important parameter of our model. The expected returns risk premium $E[R_i - r_f]$ of each assets is a function as follow:
  
\begin{equation}
    \begin{split}
        E[R_i - r_f] =& \gamma_0 + \gamma_2 X_1 + \gamma_3 X_2 + ... \gamma_n X_{n+1} \\
                     =& \gamma_0 + \sum_{i=1}^n \gamma_i X_i  \\
    \end{split}
\end{equation}
  
To arrive at the coefficients for these factors, we will train our model based on 250 different 72-month blocks of the data, splitting 60:12, i.e. 60-month data will be used as trainning set for the following 12-month data.  

The input parameters for our model are:  
1) Expected Monthly Returns for each funds.  
2) Monthly data of 14 Factors indicated above.  
3) Expected Returns for each funds based on monthly data.  
4) Standard Deviation for each funds based on monthly data.  

In [69]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

## Data Importing  

First order of business is to import the historical performance data of the assets we are trying to allocate:

In [70]:
import pandas as pd
import numpy as np

# Load the Excel Sheet
fn = r'1_data_practice.xls'

xl = pd.ExcelFile(fn)

dfs = {sh:xl.parse(sh) for sh in xl.sheet_names}  # Read each sheets to a dict

# Assign each sheet (dict) to a separate dataframe
dfReturns = dfs['returns']
dfChar = dfs['characteristics']
dfFutR = dfs['future_returns'] 

# Clean up characteristic sheet
dfChar = dfChar.T  # Transpose data so funds' factors end up in rows
dfChar.columns = dfChar.iloc[0]  # Set name for columns to be factor name
dfChar.drop(dfChar.index[0], inplace=True)  # Dropping unnecessary index row due to tranpose
dfChar

Unnamed: 0,Market capitalization,Book-to-market ratio,Past 2 to 12 month return,Past 1 month return,Past 13 to 60 month return,Stock return variance,Operating profitability,Investment,Accruals,CAPM beta,Net share issuance
Fund 1,0.2,-0.35,0.06,0.006,0.15,0.5,-0.075,0.075,0.0,0.15,0.06
Fund 2,0.734,-0.0076,0.3152,-0.201,0.0472,0.0335,0.067,-0.131,0.0,0.0,-0.0061
Fund 3,1.0,-0.15,0.05,0.0,0.05,0.0,0.0,0.5,0.5,0.0,0.1
Fund 4,-0.6,-0.6,0.12,0.012,0.3,0.0,-0.15,0.15,0.0,0.0,0.12
Fund 5,-0.2,-0.13,-0.05,0.0,0.03,-0.01,-0.02,0.48,0.0,0.0,0.09
Fund 6,0.6,0.5,-0.1,-0.01,-0.25,0.0,0.125,-0.125,-0.3,0.0,-0.1
Fund 7,0.198,-0.0472,0.335,0.0,0.0603,-0.1444,0.067,0.067,0.0,0.0076,0.0335
Fund 8,-0.6,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fund 9,1.0,0.0,0.0,-0.3,0.0,0.0,0.0,0.0,0.5,0.0,0.0
Fund 10,0.8,0.15,-0.06,0.0,-0.3,0.0,-0.06,-0.06,0.5,0.0,-0.03


Next, we inspect the data and calculate the variance-covariance matrix.

In [71]:
histR = dfReturns.loc[:, slice("Fund 1", "Fund 10")]  # Slice the returns only to grab vcov
histR.head()
vcovHistRMat = np.cov(histR)  # Get variance-covariance matrix of historical returns
vcovHistRMat

,Fund 1,Fund 2,Fund 3,Fund 4,Fund 5,Fund 6,Fund 7,Fund 8,Fund 9,Fund 10
0,0.058101,0.051248,0.012660,0.0851,0.05525,0.040203,0.052053,0.0608,0.010410,0.038908
1,-0.005652,0.020287,-0.012105,0.0089,0.00325,0.003717,0.010191,0.0125,-0.013855,-0.014919
2,0.060428,0.103077,0.056875,0.0671,0.04900,0.050768,0.068224,0.0353,0.048476,0.079825
3,0.073637,0.050137,0.026112,0.0984,0.07090,0.059703,0.077295,0.0975,0.030363,0.048093
4,-0.020262,-0.020055,-0.009130,-0.0101,-0.01295,-0.017178,-0.007962,0.0106,-0.001930,-0.029149


array([[ 5.00434815e-04,  1.81988459e-04,  5.66166859e-05, ...,
         3.62474279e-05,  2.32910327e-04,  1.02777761e-05],
       [ 1.81988459e-04,  1.50344335e-04,  5.75467294e-05, ...,
         8.43922268e-05,  2.17615607e-04, -1.20741107e-05],
       [ 5.66166859e-05,  5.75467294e-05,  3.65490203e-04, ...,
         1.06233355e-04,  9.83572167e-05, -1.77087493e-05],
       ...,
       [ 3.62474279e-05,  8.43922268e-05,  1.06233355e-04, ...,
         3.14719151e-04,  4.39406304e-04, -2.01622910e-05],
       [ 2.32910327e-04,  2.17615607e-04,  9.83572167e-05, ...,
         4.39406304e-04,  9.13129750e-04, -8.90484133e-05],
       [ 1.02777761e-05, -1.20741107e-05, -1.77087493e-05, ...,
        -2.01622910e-05, -8.90484133e-05,  3.08810724e-05]])

In [72]:
dfReturns.head()

,Unnamed: 0,RMRF,RSMB,RHML,RF,Fund 1,Fund 2,Fund 3,Fund 4,Fund 5,Fund 6,Fund 7,Fund 8,Fund 9,Fund 10
0,Month 1,0.017674,0.045959,-0.006031,0.005199,0.058101,0.051248,0.012660,0.0851,0.05525,0.040203,0.052053,0.0608,0.010410,0.038908
1,Month 2,-0.016806,0.016930,0.005840,0.005591,-0.005652,0.020287,-0.012105,0.0089,0.00325,0.003717,0.010191,0.0125,-0.013855,-0.014919
2,Month 3,0.051047,0.002810,-0.011053,0.005676,0.060428,0.103077,0.056875,0.0671,0.04900,0.050768,0.068224,0.0353,0.048476,0.079825
3,Month 4,0.037620,0.050919,-0.004756,0.005603,0.073637,0.050137,0.026112,0.0984,0.07090,0.059703,0.077295,0.0975,0.030363,0.048093
4,Month 5,-0.014375,-0.004080,0.018704,0.006325,-0.020262,-0.020055,-0.009130,-0.0101,-0.01295,-0.017178,-0.007962,0.0106,-0.001930,-0.029149


In [73]:
# Add more factor to the factor table
dfChar["RMRF"] = dfReturns["RMRF"].mean()  # Initialize expected market returns as a factor
dfChar["RSMB"] = dfReturns["RSMB"].mean()  # Intialize SMB as a factor
dfChar["RHML"] = dfReturns["RHML"].mean()  # Initalize HML as a factor

# Add E[R_i - r_f] for each fund the lazy way, there's probably better way to do this
# We will assume E[R_i] = mean(E[R_{i,j}]) where j indicates the historical month index
dfChar.at["Fund 1", "E_RPRF"] = dfReturns["Fund 1"].mean() - dfReturns["RF"].mean()
dfChar.at["Fund 2", "E_RPRF"] = dfReturns["Fund 2"].mean() - dfReturns["RF"].mean()
dfChar.at["Fund 3", "E_RPRF"] = dfReturns["Fund 3"].mean() - dfReturns["RF"].mean()
dfChar.at["Fund 4", "E_RPRF"] = dfReturns["Fund 4"].mean() - dfReturns["RF"].mean()
dfChar.at["Fund 5", "E_RPRF"] = dfReturns["Fund 5"].mean() - dfReturns["RF"].mean()
dfChar.at["Fund 6", "E_RPRF"] = dfReturns["Fund 6"].mean() - dfReturns["RF"].mean()
dfChar.at["Fund 7", "E_RPRF"] = dfReturns["Fund 7"].mean() - dfReturns["RF"].mean()
dfChar.at["Fund 8", "E_RPRF"] = dfReturns["Fund 8"].mean() - dfReturns["RF"].mean()
dfChar.at["Fund 9", "E_RPRF"] = dfReturns["Fund 9"].mean() - dfReturns["RF"].mean()
dfChar.at["Fund 10", "E_RPRF"] = dfReturns["Fund 10"].mean() - dfReturns["RF"].mean()

# Add actual R_i - r_f based on "future" returns
dfChar.at["Fund 1", "RPRF"] = dfFutR["Fund 1"].mean() - dfFutR["RF"].mean()
dfChar.at["Fund 2", "RPRF"] = dfFutR["Fund 2"].mean() - dfFutR["RF"].mean()
dfChar.at["Fund 3", "RPRF"] = dfFutR["Fund 3"].mean() - dfFutR["RF"].mean()
dfChar.at["Fund 4", "RPRF"] = dfFutR["Fund 4"].mean() - dfFutR["RF"].mean()
dfChar.at["Fund 5", "RPRF"] = dfFutR["Fund 5"].mean() - dfFutR["RF"].mean()
dfChar.at["Fund 6", "RPRF"] = dfFutR["Fund 6"].mean() - dfFutR["RF"].mean()
dfChar.at["Fund 7", "RPRF"] = dfFutR["Fund 7"].mean() - dfFutR["RF"].mean()
dfChar.at["Fund 8", "RPRF"] = dfFutR["Fund 8"].mean() - dfFutR["RF"].mean()
dfChar.at["Fund 9", "RPRF"] = dfFutR["Fund 9"].mean() - dfFutR["RF"].mean()
dfChar.at["Fund 10", "RPRF"] = dfFutR["Fund 10"].mean() - dfFutR["RF"].mean()

In [74]:
dfChar

Unnamed: 0,Market capitalization,Book-to-market ratio,Past 2 to 12 month return,Past 1 month return,Past 13 to 60 month return,Stock return variance,Operating profitability,Investment,Accruals,CAPM beta,Net share issuance,RMRF,RSMB,RHML,E_RPRF,RPRF
Fund 1,0.2,-0.35,0.06,0.006,0.15,0.5,-0.075,0.075,0.0,0.15,0.06,0.001775,0.010259,0.001611,0.000709,0.026037
Fund 2,0.734,-0.0076,0.3152,-0.201,0.0472,0.0335,0.067,-0.131,0.0,0.0,-0.0061,0.001775,0.010259,0.001611,0.008409,0.036120
Fund 3,1.0,-0.15,0.05,0.0,0.05,0.0,0.0,0.5,0.5,0.0,0.1,0.001775,0.010259,0.001611,-0.001704,0.023461
Fund 4,-0.6,-0.6,0.12,0.012,0.3,0.0,-0.15,0.15,0.0,0.0,0.12,0.001775,0.010259,0.001611,0.012194,0.035892
Fund 5,-0.2,-0.13,-0.05,0.0,0.03,-0.01,-0.02,0.48,0.0,0.0,0.09,0.001775,0.010259,0.001611,0.006888,0.032817
Fund 6,0.6,0.5,-0.1,-0.01,-0.25,0.0,0.125,-0.125,-0.3,0.0,-0.1,0.001775,0.010259,0.001611,0.007029,0.035991
Fund 7,0.198,-0.0472,0.335,0.0,0.0603,-0.1444,0.067,0.067,0.0,0.0076,0.0335,0.001775,0.010259,0.001611,0.009521,0.038377
Fund 8,-0.6,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001775,0.010259,0.001611,0.011537,0.037350
Fund 9,1.0,0.0,0.0,-0.3,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.001775,0.010259,0.001611,-0.000696,0.022516
Fund 10,0.8,0.15,-0.06,0.0,-0.3,0.0,-0.06,-0.06,0.5,0.0,-0.03,0.001775,0.010259,0.001611,0.003462,0.035842


## Optimization Functions and Constraint Functions. 

We develop the optimization function $negSharpe()$, $conSumWeight()$, and $conLimitWeight()$.

### negSharpe()  
Our objective is maximizing Sharpe ratio. Scipy Optimize Minimize optimizes the min so we add negative component to the Sharpe Ratio equation.  

### conSumWeight() 
This constraint is an equality constraint in the scipy.optmize engine, ensuring our weights sum up to 1.  

### conLimitWeight()  
This constraint is an equality constraint, ensuring that no weight have absolute value exceeding 2.

In [75]:
import numpy as np

# Calculate Sharpe Ratio
def negSharpe(wVector):
    ret = np.dot(np.squeeze(np.asarray(wVector)), np.squeeze(np.asarray(returnsVector))) - mean_rf
    risk = np.matmul(np.matmul(wVector, covMatrix), np.transpose(wVector))
    return -(ret - mean_rf) / risk 

# Add sum weights = 1 constraint
def conSumWeight(wVector):
    return np.sum(wVector) - 1

# Add abs|weight| <= 2
def conLimitWeight(wVector):    
    return - abs(wVector) + 2

### Weights Optimization

Reviewing the fitted intercept and coefficient for asset risk premium

In [76]:
# Reload data from Data Processing Engine
%store -r intercept
%store -r coefArr

eReturnsIntercept = intercept
eReturnsCoefs = coefArr

eReturnsIntercept
eReturnsCoefs

0.005892826321073623

array([-5.05445103e-04,  9.07507789e-04,  2.46796015e-03, -7.39882495e-04,
       -3.21815971e-04, -7.55341655e-04, -5.15029443e-05, -2.94201231e-04,
       -2.80309888e-04,  2.10692659e-04, -3.06316569e-04,  4.97911042e-03,
        3.53742536e-03,  2.65027253e-04])

Where the magic happens 😯😯😯😯😯😯

In [77]:
import scipy.optimize as optimize

# Generate initial array of guess weight, all .1 or 10% for each of the 10 funds
initial_guess = np.full((1,10), .1)

# Set up covariance matrix from downloaded data
global covMatrix 
covMatrix = np.cov(dfReturns.loc[:,"Fund 1": "Fund 10"].T)

# Set up E[r_f]
global mean_rf
mean_rf = dfReturns["RF"].mean()

# Set up factor matrix
factorMat = dfChar.loc[:,"Market capitalization": "RHML"]

# Set up E[R_i] vector
global returnsVector
returnsVector = np.dot(eReturnsCoefs, factorMat.T) + np.full((1,10), eReturnsIntercept) + np.full((1,10), mean_rf)

# Set up constraints
cons = [{'type': 'eq', 'fun': conSumWeight}, 
        {'type': 'ineq', 'fun': conLimitWeight}]

# Optimize!
result = optimize.minimize(negSharpe, initial_guess, constraints=cons)

## Results 

Examine our result:

In [78]:
result

     fun: -0.9226402122223921
     jac: array([-4.1214058 , -4.16385391, -4.16416752, -4.16422996, -4.05107338,
       -4.16404272, -4.2674559 , -4.18702248, -4.16443913, -4.1641038 ])
 message: 'Optimization terminated successfully'
    nfev: 278
     nit: 25
    njev: 25
  status: 0
 success: True
       x: array([-2.        ,  1.48028032, -0.20912529, -0.34140553, -2.        ,
        0.25527652,  2.        ,  2.        ,  0.62421287, -0.80923889])

### Optimized Weights  

We can see our optimize weight array to be:  

In [79]:
idx = pd.Index(["Fund 1", 
                       "Fund 2", 
                       "Fund 3", 
                       "Fund 4", 
                       "Fund 5", 
                       "Fund 6", 
                       "Fund 7", 
                       "Fund 8", 
                       "Fund 9", 
                       "Fund 10"])
optimizedResult = pd.DataFrame(result.x, columns=["Weight"]).set_index(idx)
optimizedResult

,Weight
Fund 1,-2.000000
Fund 2,1.480280
Fund 3,-0.209125
Fund 4,-0.341406
Fund 5,-2.000000
Fund 6,0.255277
Fund 7,2.000000
Fund 8,2.000000
Fund 9,0.624213
Fund 10,-0.809239


### Constraint check  
We can see that the weight constraint are satisfied and visually, the upper and lower limit of 200% also satisfied:

In [80]:
result.x.sum()

1.0

### Sharpe Ratio  

The Sharpe Ratio is estimated to be:

In [81]:
abs(result.fun)

0.9226402122223921

### Portfolio Returns  

The Portfolio Returns is:  

In [82]:
np.dot(np.squeeze(np.asarray(result.x)), np.squeeze(np.asarray(returnsVector)))

0.019135088956192946

### Portfolio Risk  

The Portfolio Risk is

In [83]:
np.matmul(np.matmul(result.x, covMatrix), np.transpose(result.x))

0.0029110985393748998